In [8]:
import torch
import numpy as np
import pandas as pd
# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device =  -1
print(device)

0


In [9]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=device)

In [10]:
import pandas as pd
labels = open('data/classes.txt').read().splitlines()
all_df = pd.read_csv("data/belief_benchmark.csv")
all_df

,text,sentiment,label
0,Also use of chemicals and machinery on their p...,"NEGATIVE, NEGATIVE",1
1,Sources believe that twenty-five to 30 percent...,UNDETERMINED,0
2,Although these government-led initiatives are ...,UNDETERMINED,0
3,"In other projects of this kind, it is not alwa...",POSITIVE,2
4,Many African policy makers evidently believe t...,POSITIVE,2
...,...,...,...
358,Similar initiatives have and can be initiated ...,POSITIVE,2
359,Collective sales account for an average of 64 ...,UNDETERMINED,0
360,Individual sales account for slightly lower vo...,"UNDETERMINED, UNDETERMINED, UNDETERMINED",0
361,Modernization of the rice value chain The VC t...,UNDETERMINED,0


In [ ]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['test'] = Dataset.from_pandas(all_df)

In [13]:
# import numpy as np

# labels = ["neither", "negative", "positive", "both"]
# pred_i_list = []
# for i,row in all_df.iterrows():
#     pred = classifier(row["text"], candidate_labels=labels)
#     pred_i = labels.index(pred["labels"][0])
#     pred_i_list.append(pred_i)

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\transformers\pipelines\base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


KeyboardInterrupt: 